# Training the bert model

## installing libraries

In [ ]:
!pip install transformers pandas numpy matplotlib tqdm keras
!pip install lightgbm --install-option=--gpu


### Imports

In [ ]:
# general libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import random

random.seed(42)
np.random.seed(42)

In [ ]:
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

from lightgbm import LGBMClassifier

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
for l,v in {'pandas':pd, 'numpy':np, 'sklearn':sklearn,'transformers':transformers}.items():
    print(f'{l}  version {v.__version__}')

pandas  version 1.1.3
numpy  version 1.18.5
keras  version 2.4.3
sklearn  version 0.22.2.post1
transformers  version 3.4.0


## Reading data from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
baseDir = '/content/gdrive/My Drive/Colab Notebooks/AA'

In [ ]:
datasets = pd.read_json(baseDir+'/data/AllDS_BERT.json.zip', orient='records', compression='gzip')
datasets['row_index'] = np.arange(len(datasets));
datasets = datasets[['row_index','dataset','problem','language','set','filename','text','label','BERT_vector']]


*   **row_index**: auxiliaray field to help merging the result of the models.
*   **Dataset**: group of documents belonging to the same scenario (social, literature, lyrics, etc)
*   **Problem**:  a especific test case like 20 authors with short text or 5 authors with long texts.
*   **Set**: Known is the development set,  unkown is the validation. Note this is not the train-test split, what should be done only with the development set.
*   **Filename**: the original filename in the corpus
*   **label**: the target
*   **BERT_vector**: the text corresponding to bert vector



In [ ]:
datasets.head()

,row_index,dataset,problem,language,set,filename,text,label,BERT_vector
0,0,pan18_train,problem00001,en,known,known00001.txt,"graceful ones.\n\n""One more,"" Marvelous said, ...",candidate00001,"[0.4001088142, -0.9896771312, -0.2849564552, -..."
1,1,pan18_train,problem00001,en,known,known00002.txt,"before. If he can, he’ll remember a classmate ...",candidate00001,"[-0.1036661863, -0.2112793922, -0.1762309372, ..."
2,2,pan18_train,problem00001,en,known,known00003.txt,she thought - he was in Team Baron only becaus...,candidate00001,"[0.1540681422, -0.3122391403, -0.0233840719, -..."
3,3,pan18_train,problem00001,en,known,known00007.txt,"As far as she remembers, she's always hated pr...",candidate00001,"[-0.013924662, -0.4036496282, -0.3019542694000..."
4,4,pan18_train,problem00001,en,known,known00006.txt,"“Wait for me, please!”\n\nShe glanced towards ...",candidate00001,"[-0.1647676378, -0.2584642172, 0.1031820625, 0..."


In [ ]:
datasets.groupby(['dataset']).agg({'problem':'nunique'}).T

dataset,lyrics,pan18_eval,pan18_train,socialaa
problem,10,20,10,32


## Trainining the models with kfold  cross validation

In [ ]:
clfs = {
    'LR': make_pipeline(MinMaxScaler(),LogisticRegression(max_iter=1000, C=0.1, random_state=42)),
    'LGBM': make_pipeline(MinMaxScaler(), LGBMClassifier(max_depth=7,n_estimators=25, num_leaves=32)),
    'MLP': make_pipeline(MinMaxScaler(),MLPClassifier(hidden_layer_sizes=[15,15],max_iter=700, random_state=42,early_stopping=True)),
}

In [ ]:
best_estimators = [];
predictions = [];
classification_reports = [];

#for each dataset, for each problem, for each classifier, run cross validation
for dataset in datasets['dataset'].unique():
    for problem in datasets.query(f"dataset == '{dataset}'")['problem'].unique():
        print('\n',dataset,problem,' ---- ',end=' ');

        X_train, y_train, index_train, X_test, y_test, index_test = filter_dataset(datasets,dataset,problem)

        embedding_size = X_train.shape[1];
        nclass = len(np.unique(y_train));

        for c in clfs:
            print(c, end=' ');
            classes_ = np.sort(np.unique(y_train))
            grid_search = GridSearchCV(estimator = clfs[c],param_grid={}, scoring = 'f1_macro', cv = 5)
            grid_search.fit(X_train, y_train);
            clf = grid_search.best_estimator_;

            pred_train = clf.predict(X_train);
            pred_proba_train = clf.predict_proba(X_train);

            pred_test = clf.predict(X_test);
            pred_proba_test = clf.predict_proba(X_test);

            #saving predictions for future analysis
            def appendPrediction(predictions,classes_,index, pred,proba):
                for i,p, pr in  zip(index, pred, proba):
                    predictions.append(dict(**{
                            'dataset':dataset,
                            'problem':problem,
                            'model':c,
                            'row_index':i,
                            'pred':p,
                        }, **{
                            cc:pr_ for cc,pr_ in zip(classes_, pr)
                        })
                    )

            appendPrediction(predictions,clf.classes_, index_train, pred_train, pred_proba_train)
            appendPrediction(predictions,clf.classes_, index_test, pred_test, pred_proba_test)


            #saving estimators
            best_estimators.append({
                'dataset':dataset,
                'problem':problem,
                'model':c,
                'estimator':clf
            })

            class_report = metrics.classification_report(y_test,pred_test,output_dict=True)

            classification_reports.append(dict(**{
                'dataset':dataset,
                'problem':problem,
                'model':c,
                'classification_report':class_report
            }, **class_report['macro avg']))
        print();


 pan18_train problem00001  ----  LR LGBM MLP 

 pan18_train problem00002  ----  LR LGBM MLP 

 pan18_train problem00003  ----  LR LGBM MLP 

 pan18_train problem00004  ----  LR LGBM MLP 

 pan18_train problem00005  ----  LR LGBM MLP 

 pan18_train problem00006  ----  LR LGBM MLP 

 pan18_train problem00007  ----  LR LGBM MLP 

 pan18_train problem00008  ----  LR LGBM MLP 

 pan18_train problem00009  ----  LR LGBM MLP 

 pan18_train problem00010  ----  LR LGBM MLP 

 pan18_eval problem00001  ----  LR LGBM MLP 

 pan18_eval problem00002  ----  LR LGBM MLP 

 pan18_eval problem00003  ----  LR LGBM MLP 

 pan18_eval problem00004  ----  LR LGBM MLP 

 pan18_eval problem00005  ----  LR LGBM MLP 

 pan18_eval problem00006  ----  LR LGBM MLP 

 pan18_eval problem00007  ----  LR LGBM MLP 

 pan18_eval problem00008  ----  LR LGBM MLP 

 pan18_eval problem00009  ----  LR LGBM MLP 

 pan18_eval problem00010  ----  LR LGBM MLP 

 pan18_eval problem00011  ----  LR LGBM MLP 

 pan18_eval problem0001

## Saving the classification results for future analysis

In [ ]:
#saving estimators
import pickle
import bz2
with bz2.BZ2File(baseDir+'/BERT/output_bert/bert_best_estimators.pkl.bz2','wb') as f:
    pickle.dump(best_estimators,f);

In [ ]:
pd.DataFrame(predictions)\
    .sort_values('row_index').round(5)\
    .to_csv(baseDir+'/BERT/output_bert/bert_predictions.csv.zip', index=False, compression='zip', encoding='utf-8')

### predictions 

* predictions file contains the prediction for a dataset|problem|instance vs LR|MLP|LGBM
* Model is the model pipeline.
* Prediction is the label with the highest probability

In [ ]:
pd.DataFrame(predictions).round(5).sort_values('row_index')

,dataset,problem,model,row_index,pred,candidate00001,candidate00002,candidate00003,candidate00004,candidate00005,candidate00006,candidate00007,candidate00008,candidate00009,candidate00010,candidate00011,candidate00012,candidate00013,candidate00014,candidate00015,candidate00016,candidate00017,candidate00018,candidate00019,candidate00020,candidate00021,candidate00022,candidate00023,candidate00024,candidate00025,candidate00026,candidate00027,candidate00028,candidate00029,candidate00030,candidate00031,candidate00032,candidate00033,candidate00034,candidate00035,candidate00036,candidate00037,candidate00038,candidate00039,candidate00040,candidate00041,candidate00042,candidate00043,candidate00044,candidate00045,candidate00046,candidate00047,candidate00048,candidate00049,candidate00050
0,pan18_train,problem00001,LR,0,candidate00001,0.31580,0.02728,0.01090,0.03243,0.04595,0.03042,0.04867,0.01975,0.04859,0.05737,0.03326,0.01372,0.02993,0.07244,0.02539,0.03529,0.03982,0.03997,0.03499,0.03804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,pan18_train,problem00001,LGBM,0,candidate00001,0.48454,0.02085,0.01822,0.02566,0.01875,0.03488,0.02635,0.02891,0.01893,0.06259,0.01025,0.01134,0.01335,0.07360,0.02828,0.01434,0.04600,0.01904,0.02012,0.02400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
490,pan18_train,problem00001,MLP,0,candidate00005,0.06086,0.03304,0.06282,0.03827,0.06884,0.04990,0.05381,0.03692,0.05603,0.04218,0.05754,0.05448,0.04543,0.05300,0.04507,0.06163,0.02969,0.03630,0.05645,0.05773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
491,pan18_train,problem00001,MLP,1,candidate00005,0.06837,0.04095,0.05835,0.03500,0.08937,0.06533,0.04328,0.03291,0.06356,0.04717,0.05221,0.04062,0.04202,0.05834,0.04477,0.06412,0.02353,0.03138,0.05032,0.04840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pan18_train,problem00001,LR,1,candidate00001,0.34838,0.06337,0.01064,0.05981,0.04476,0.01041,0.02305,0.01504,0.04484,0.03527,0.04471,0.01362,0.05181,0.05797,0.03475,0.05394,0.02444,0.02972,0.01716,0.01631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92896,socialaa,problem00032,MLP,30964,candidate00031,0.00399,0.00785,0.00001,0.00001,0.00009,0.03388,0.01107,0.00142,0.00078,0.09011,0.00130,0.06559,0.00043,0.00239,0.00000,0.00012,0.11235,0.00003,0.06147,0.00449,0.00001,0.00000,0.00428,0.08226,0.00030,0.00348,0.01666,0.02823,0.01296,0.00038,0.11860,0.00073,0.04028,0.01851,0.00254,0.00298,0.00000,0.00000,0.07581,0.00239,0.00126,0.00618,0.00484,0.00231,0.00011,0.00035,0.10595,0.00006,0.07113,0.00002
87796,socialaa,problem00032,LR,30964,candidate00049,0.00265,0.00362,0.00089,0.01404,0.00422,0.00811,0.02147,0.00913,0.00230,0.02881,0.01582,0.06632,0.00078,0.01912,0.00449,0.00497,0.02752,0.00133,0.02165,0.00071,0.00031,0.00719,0.00034,0.02887,0.04554,0.00386,0.00462,0.14114,0.00809,0.00187,0.03274,0.00180,0.00398,0.00969,0.01540,0.01052,0.00131,0.00031,0.06494,0.02468,0.00353,0.00213,0.00823,0.00153,0.00704,0.00294,0.01739,0.00554,0.28495,0.00155
90347,socialaa,problem00032,LGBM,30965,candidate00044,0.00939,0.00938,0.00939,0.00939,0.01968,0.02922,0.01054,0.00942,0.00939,0.00939,0.01343,0.01143,0.00941,0.00937,0.00940,0.01715,0.01273,0.00940,0.00940,0.01146,0.00940,0.00938,0.01644,0.00944,0.01955,0.01133,0.01212,0.00939,0.02304,0.00938,0.01508,0.00936,0.01389,0.00956,0.00939,0.00940,0.00940,0.00940,0.00940,0.00938,0.00942,0.01426,0.01199,0.22971,0.05096,0.01314,0.00941,0.02584,

### Classification Reports

The classification report contains the results for the test (or unknown text, or  validation set) which is the official result.

In [ ]:
pd.DataFrame(classification_reports).to_json(
    baseDir+'/BERT/output_bert/bert_classification_reports.json.zip',
    orient='records',
    compression='zip'
)

In [ ]:
import re
def statistics(x):
    docs = x.query('set == "known"').groupby('label').agg({'filename':'nunique'}).mean().astype(int).values[0];

    nchar = int(x.query('set == "unknown"')['text'].apply(lambda x:len(x)).mean());
    leastOne = lambda x: x if x>0 else 1;
    nword = x.query('set == "unknown"')['text'].apply(lambda x:len(re.findall(r'\b\w+\b',x))).apply(leastOne).mean();

    nauthors = len(x['label'].unique())
    return pd.Series({
        'ndocs':docs,
        'nauthors':nauthors,
        'nchar': int(nchar/10)*10,
        'nword':int(np.ceil(nword/5)*5),
    })

metadata = datasets.groupby(['dataset','problem','language']).apply(statistics).reset_index()

In [ ]:
temp = pd.DataFrame(classification_reports) \
        .pivot_table(
            index=['dataset','problem'],
            columns='model',
            values='f1-score')[['LR','LGBM','MLP']]

temp2 = temp.reset_index().merge(metadata)

with open(baseDir+'/BERT/output_bert/bert_report.txt','w') as f:

    f.write(temp.round(2).to_latex())
    f.write("\n\n");
    f.write(temp2.groupby(['dataset','language']).mean()[['LR','LGBM','MLP']].reset_index().round(2).to_latex(index=False))

    f.write("\n\n");
    f.write(temp2.groupby(['dataset','ndocs']).mean()[['LR','LGBM','MLP']].reset_index().round(2).to_latex(index=False))

    f.write("\n\n");
    f.write(temp2.groupby(['dataset','nauthors']).mean()[['LR','LGBM','MLP']].reset_index().round(2).to_latex(index=False))

    f.write("\n\n");
    f.write(temp2.groupby(['dataset','nchar']).mean()[['LR','LGBM','MLP']].reset_index().round(2).to_latex(index=False))

    f.write("\n\n");
    f.write(temp2.groupby(['dataset','nword']).mean()[['LR','LGBM','MLP']].reset_index().round(2).to_latex(index=False))


In [ ]:
with pd.option_context("display.max_rows", 100):
    display(temp.round(4))

model                         LR    LGBM     MLP
dataset     problem                             
lyrics      problem00001  0.4302  0.3670  0.1733
            problem00002  0.1585  0.1477  0.0292
            problem00003  0.1347  0.1390  0.0361
            problem00004  0.1066  0.0860  0.0106
            problem00005  0.0767  0.0877  0.0060
            problem00006  0.4240  0.3284  0.0718
            problem00007  0.2456  0.2635  0.0333
            problem00008  0.1905  0.2056  0.0091
            problem00009  0.1968  0.1771  0.0026
            problem00010  0.1118  0.1219  0.0157
pan18_eval  problem00001  0.2107  0.0969  0.0281
            problem00002  0.2540  0.1426  0.0304
            problem00003  0.4288  0.2111  0.0103
            problem00004  0.4000  0.0235  0.1400
            problem00005  0.1142  0.1024  0.0187
            problem00006  0.1112  0.0758  0.0226
            problem00007  0.1558  0.1721  0.0000
            problem00008  0.3421  0.0960  0.0222
            problem00009  0.3522  0.2478  0.0244
            problem00010  0.3186  0.2813  0.0314
            problem00011  0.5064  0.3964  0.0679
            problem00012  0.1332  0.0098  0.1662
            problem00013  0.0691  0.0605  0.0008
            problem00014  0.0759  0.0136  0.0016
            problem00015  0.0939  0.0250  0.0212
            problem00016  0.1737  0.0780  0.0364
            problem00017  0.1177  0.0788  0.0070
            problem00018  0.1197  0.1094  0.0149
            problem00019  0.1884  0.1079  0.0157
            problem00020  0.4333  0.0296  0.0870
pan18_train problem00001  0.1110  0.0675  0.0097
            problem00002  0.4800  0.0769  0.1375
            problem00003  0.2200  0.1200  0.0354
            problem00004  0.5800  0.0640  0.2379
            problem00005  0.4180  0.2323  0.0024
            problem00006  0.4840  0.0933  0.1057
            problem00007  0.0403  0.0620  0.0191
            problem00008  0.1723  0.1500  0.0267
            problem00009  0.1918  0.1499  0.0039
            problem00010  0.3400  0.0988  0.0915
socialaa    problem00001  0.1333  0.1333  0.0667
            problem00002  0.4667  0.4667  0.0667
            problem00003  0.2333  0.1667  0.1286
            problem00004  0.3500  0.1667  0.0182
            problem00005  0.1167  0.1000  0.0062
            problem00006  0.1533  0.2667  0.0048
            problem00007  0.1167  0.0880  0.0009
            problem00008  0.0647  0.0667  0.0147
            problem00009  0.7333  0.2667  0.0667
            problem00010  0.0000  0.0000  0.0667
            problem00011  0.7333  0.3333  0.2800
            problem00012  0.2333  0.5333  0.0667
            problem00013  0.5000  0.7333  0.2333
            problem00014  1.0000  0.7333  0.0800
            problem00015  0.4000  0.3000  0.0182
            problem00016  0.4167  0.3667  0.0222
            problem00017  0.6333  0.4167  0.0333
            problem00018  0.5333  0.3067  0.0917
            problem00019  0.6333  0.7333  0.0200
            problem00020  0.2400  0.2167  0.0733
            problem00021  0.2533  0.2310  0.0050
            problem00022  0.3617  0.4083  0.0048
            problem00023  0.6667  0.3667  0.0050
            problem00024  0.2750  0.2450  0.0059
            problem00025  0.3083  0.2700  0.0091
            problem00026  0.5500  0.6000  0.0556
            problem00027  0.0713  0.1467  0.0041
            problem00028  0.2333  0.2267  0.0009
            problem00029  0.3567  0.2233  0.0008
            problem00030  0.1020  0.1733  0.0017
            problem00031  0.2833  0.1667  0.0130
            problem00032  0.5000  0.3013  0.1033